# **Business Cases Course**

<br>

- **Group Project 2022/2023**
- **Academic Year: 2020-2023 | 2nd Semester**
- **Professor: Nuno António**

<br>

- **"Case 2: Market Basket Analysis"**
- **Asian Food Restaurant from Company C**
- **This notebook uses the *Case2_AsianRestaurant_Cyprus_2018.txt* dataset**


<br>

> **Group composed by:**<p>
> Ana Carolina Ottavi, nº 20220541<p>
> Carolina Bezerra, nº 20220392 <p>
> Duarte Girão, nº 20220670<p>
> João Pólvora, nº 20221037<p>
> Luca Loureiro, nº 20221750<p>

## 📖 Introduction
    
Within the scope of __Business Cases for Data Science__, it was proposed a project, where the groups' ability to deliver a market basket analysis in accordance with the different features included in a dataset containing 84.109 records would be tested. However, the company C that ownes this restaurant realized that they are not having enough profits and therefore, wishes to better understand their customers and its preferences.
    
Therefore, the business goal determined for this project was to develop a market basket analysis. C expects to gain insights in terms of creation of menus, introduction of new products, understand substitute products, recommending/promoting cross-selling, customer segmentation and other possible results depending on the findings. 

C has several questions that it hopes to get answers to:
- Are there any differences between dine-inn customers and delivery customers?
- Is the product offering adequate (e.g., do customers make strange combination of products) ?
- Are there any patterns in consumption that may indicate tendencies?

<br>

## 📖Dataset description

This notebook uses the Case2_AsianRestaurant_Cyprus_2018.txt. The Dataset is related with all sales transactions of a restaurant located in Nicosia, the capital of Cyprus.<br> This asian food restaurant is inserted in a company C, with restaurants spread all over the this country. <br> This restaurant is struggling to maintain its profit margin and continuous growth due to increasing competition and customers’ changes in habits. To try to revert this process, C wants to take advantage of its sales data to understand customers’ patterns of consumption and preferences.

The dataset is composed of the following columns:
- **DocNumber**: number of the document. The document number repeats in as many rows as the rows in the original document (invoice)
- **ProductDesignation**: product designation
- **ProductFamily**: name of the family of the product. A product can only be member of one only family
- **Qty**: quantity
- **TotalAmount**: sale price of the total quantity
- **InvoiceDateHour**: date and hour when the document was issued
- **EmployeeID**: ID of the employee who issued the document
- **IsDelivery**: indication if sale was a delivery or a dine-inn (1:
delivery, 0: dine-inn)
- **Pax**: number of persons at the table
- **CustomerID**: ID of the customer (if a customer record was
assigned to the sale)
- **CustomerCity**: city of the customer (usually only employed in
delivery)
- **CustomerSince**: date of creation of the customer

Note: Each row in the dataset represents a document line (invoice line).

# **1. Initial Setup**

- 1.1) Importing needed libraries
- 1.2) Defining functions

In [1]:
# Install in case its necessary
    # pip install mlxtend
    # pip install networkx
    # pip install WorldCloud
    # pip install geopy
    # pip install folium
    
# Imports
import category_encoders as ce
import datetime as dt
import folium
import joypy
import itertools
import math
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import missingno as msno
import networkx as nx
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import re
import seaborn as sns
import scipy.stats as stats
import warnings
import ydata_profiling
warnings.filterwarnings("ignore")

# From mlxtend
# Associated Rules related imports
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# From other libraries
from datetime import datetime, timedelta
from dateutil import parser
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import exc
from itertools import cycle,islice, product
from math import ceil
from matplotlib import ticker
from plotly.subplots import make_subplots
from os.path import join
from ydata_profiling import ProfileReport
from wordcloud import WordCloud

# Setting Visual Theme
sns.set_theme(style = 'white', palette = None)

# **2. Data Understanding**

- **2.1) Collect Initial Data: Reading the original file**
- 2.1.1) Reading the original file
- 2.1.2) Making a safety copy of the dataset
- **2.3) Describe Data: General description of data**
- **2.4) Explore Data**
- 2.4.1) Counting the number of rows and columns 
- 2.4.2) Checking the top and bottom from the dataset
- 2.4.3) Checking Data types
- 2.4.4) Describe Data: General description of data after changing dtypes
- 2.4.5) Checking the number of unique values per feature
- **2.6) Verify Data Quality**
- 2.6.1) Checking Duplicated Values
- 2.6.2) Checking Missing Values
- 2.6.3) Checking CustomerSince and InvoiceDateHour
- 2.6.4) Checking DocNumber
- 2.6.5) Checking ProductDesignation and ProductFamily
- 2.6.6) Checking Qty and Total Amount
- 2.6.7) Checking EmployeeID
- 2.6.8) Checking Pax and CustomerID

- **2.1) Collect Initial Data: Reading the original file**
- 2.1.1) Reading the original file

In [2]:
'''
dtypes = {'DocNumber':'category',
          'ProductDesignation':'category', 
          'ProductFamily':'category',
          'Qty':'category', 
          'TotalAmount': 'category',
          'InvoiceDateHour':'category',
          'EmployeeID':'category',
          'IsDelivery':'category',
          'Pax':'category',
          'CustomerID':'category',
          'CustomerCity':'category',
          'CustomerSince':'category'}
df = pd.DataFrame(pd.read_csv("Case2_AsianRestaurant_Cyprus_2018.txt",sep=";", dtype=dtypes))'''

# Reading the dataframe.
df = pd.DataFrame(pd.read_csv("C:\\Users\\cmade\\OneDrive\\Ambiente de Trabalho\\BC2",sep=";"))
df.info()

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\cmade\\OneDrive\\Ambiente de Trabalho\\BC2'

- **2.1) Collect Initial Data: Reading the original file**
- 2.1.2) Making a safety copy of the dataset

In [ ]:
# Make a first security copy of our original dataset.

df_original = df.copy()

- **2.3) Describe Data: General description of data**

In [ ]:
# Check descriptive statistics before replacing missing values.
# Here, we can have a good sensibility on data.

df.describe(include="all").T

**Key Takeawyays**
- There are repeated DocNumbers, which is possible since each DocNumber is repeated for how many products there are in the invoice. The restaurant has 11.147 invoices for the year of 2018.
- There are 255 different product designations and 27 different product families. Mineral water was the most requested product (7061 times). Starters is the most requested family of products (14.148 times).
- Most people order 1 unit of each product.
- The most ordered products have a cost of 3 to the customer over the total quantity. However there's a wrong ponctuation for this column - there shouldnt be a comma, instead, it should be a dot.
- The most busy day in terms of hours is the 24th of December of 2018, which is the Christmas Eve. Makes sense that more people come in the holidays.
- There isn't an employee 1 in the restaurant and theres an employee 27. Employee 2 takes the most orders
- Most orders are for dine-in
- The majority of tables answered have only 1 person and the maximum value for pax is 200
- Most customers don't have an ID
- Customers registered are from 17 different cities and most cusotmers are from Egkomi, which is a suburb and municipality of the Cypriot capital Nicosia
- The customers of this restaurant have been customers of it for a long while

In [ ]:
# # Visualizing the profilling of our dataset to have a general sensibility on data.
# df_profilling =ydata_profiling.ProfileReport(df)
# partial_df_profilling =ydata_profiling.ProfileReport(partial_df)
# df_profilling. to_file("df_report.html")
# partial_df_profilling. to_file("partial_df_report.html")

- **2.4) Explore Data**
- 2.4.1) Counting the number of rows and columns 

In [ ]:
# Count the number of rows and columns.
# Get more sensibility on data.
    # 84109 rows (records)
    # 12 columns (variables)

df.shape

- **2.4) Explore Data**
- 2.4.2) Checking the top and bottom from the dataset

In [ ]:
# Checking the first two on top and on the bottom of the dataset df
# Is there a difference between TK and TKD?

df_sliced = df.head(2)
df_sliced = df_sliced.append(df.tail(2))
df_sliced

- **2.4) Explore Data**
- 2.4.3) Data types

In [ ]:
# Check dataset data types.
df.dtypes

In [ ]:
# Here, we are changing the three datatypes.
# There was an error converting dtype of 'TotalAmount' from object to float because there was a comma (,)
# instead of (.) for decimals units.

df.TotalAmount = df.TotalAmount.apply(lambda x:float(str(x.replace(',','.'))))
df['InvoiceDateHour'] = pd.to_datetime(df['InvoiceDateHour'])
df['CustomerSince'] = pd.to_datetime(df['CustomerSince'])
df.info()

- **2.4) Explore Data**
- 2.4.4) Describe Data: General description of data after changing dtypes

In [ ]:
# General sensibility on data.

df.describe(include="all").T

**Key Takeaways:**
- The mean of total amount of value paid for a specific quantity of a product is around 9.83 and the max is 3000

- **2.4) Explore Data**
- 2.4.5) Checking the number of unique values per feature

In [ ]:
# Get the number of unique values per feature.
# Check each feature cardinality.

df.nunique().to_frame(name = 'Number of unique values per feature')

In [ ]:
# Get unique values for all the features

columns = df.columns.to_list()

for column in columns:
    #print("\n")
    print("Column Name:", column)
    print(df[column].unique())
    print("\n")

- **2.6) Verify Data Quality**
- 2.6.1) Checking Duplicated Values

In [ ]:
# Check the duplicated values on our dataset.
# There 3923 duplicated records.

df.duplicated().sum()

- **2.6) Verify Data Quality**
- 2.6.2) Checking Missing Values

In [ ]:
# Represent a missing values in pandas with a NaN value.

df.replace("", np.nan, inplace=True)

In [ ]:
# Check the missing values on our dataset.

df.isna().sum()

In [ ]:
# Getting the number of null values for the group of deliveries and for the total. 

print(df.loc[df["IsDelivery"]==0, ].isnull().sum()) # Representing Dine-Inns.
print(df.loc[df["IsDelivery"]>0, ].isnull().sum()) # Representing Deliveries.

In [ ]:
# Taking a visual look at the missing values per variable.
# We can easily check that the variables 'CustomerCity' and 'CustomerSince' have a significant amount of missing values.

msno.bar(df)

- **2.6) Verify Data Quality**
- 2.6.3) Checking CustomerSince and InvoiceDateHour

In [ ]:
# InvoiceDateHour
# Checking the unique values of this variable.

df['InvoiceDateHour'].sort_values().unique()

In [ ]:
# CustomerSince
# Checking the unique values of this variable.
# This plataform exists since 2005, which is the date of the oldest client.

df['CustomerSince'].sort_values().unique(), 
print('The total amount of impossible values is:', df[df['CustomerSince']>'2018-12-31 23:59:59'].shape[0])

In [ ]:
# Under our understanding, it is impossible that CustomerSince is higher than InvoiceDateHour:
    # InvoiceDateHour: date and hour when the document was issued.
    # CustomerSince: date the customer was created.
    
df[df['InvoiceDateHour'] < df['CustomerSince']].shape[0]
print('The total amount of impossible values is:', df[df['InvoiceDateHour'] < df['CustomerSince']].shape[0])

- **2.6) Verify Data Quality**
- 2.6.4) Checking DocNumber

In [ ]:
# DocNumber
# Checking the main values amount of this variables.
# Some invoices include orders with a lot of different products

df['DocNumber'].value_counts().sort_values(ascending=False)

In [ ]:
# What's WITH Product Family?
# Getting our understanding of ProductFamily per DocNumber.

pd.crosstab(df['DocNumber'], df['ProductFamily'])

In [ ]:
# Why are there products registered that has a TotalAmount=0 or TotalAmount equal to a few cents?
# Getting our understanding of TotalAmount per DocNumber.

pd.crosstab(df['DocNumber'], df['TotalAmount'])

In [ ]:
# Now we know employee that number of employees equal to ONE doesn't exist.
# The employees working in the restaurant at 2018 are designated with the nrs. 2, 4, 5, 6, 7, 23 and 27.
# Getting our understanding of EmployeeID per DocNumber.

pd.crosstab(df['DocNumber'], df['EmployeeID'])

In [ ]:
# The fact that TK DocNumber have numbers when IsDelivery=0, 
# makes us believe that TK is code used for dine-ins and TKD is code used for deliveries.
# Getting our understanding of IsDelivery per DocNumber.

pd.crosstab(df['DocNumber'], df['IsDelivery'])

In [ ]:
# Getting our understanding of the relationship between 'IsDelivery' and 'CustomerID'.
# We can check that all the times that the 'CustomerID' has no value, means that it is a 'DineIn' and not a delivery.

df['CustomerID'] = df['CustomerID'].astype('str')
df_deliveries = pd.crosstab(index=np.where(df['CustomerID']>'0','1','0'), columns=df['IsDelivery'])
df_deliveries.index=['Invoice with no customer ID','Invoice with customer ID']
df_deliveries

- **2.6) Verify Data Quality**
- 2.6.5) Checking ProductDesignation and ProductFamily

In [ ]:
# Taking a close look at the relationship between ProductDesignation, ProductFamily and TotalAmount.
# Now, we can see that extras total amount is equal to 0, as well as holds equals to no meat. But also other products, 
# which dont make that much sense.
# We also can tell that delivery has a charge, which counts as extras

#df.groupby('TotalAmount')['ProductDesignation', 'ProductFamily'].sum()
df.groupby(['ProductDesignation', 'ProductFamily'])['TotalAmount'].sum()

In [ ]:
# Taking a close look at the relationship between ProductFamily and TotalAmount.
# Now, we can see that extras total amount is equal to 0, but also other products, which dont make that much sense

categorical_table = pd.crosstab(index=df['ProductFamily'], columns=df['TotalAmount'])
categorical_table

In [ ]:
# 'With' and 'Holds' PorductFamily have no price

concat_df = pd.concat([df.groupby(["ProductFamily"])["Qty"].sum(), 
                       df.groupby(["ProductFamily"])["TotalAmount"].sum()], axis=1)
display(concat_df)

In [ ]:
# Convert lowercase column using str.lower().
df['ProductFamily'] = df['ProductFamily'].str.lower()

# Capitalize first letter.
df['ProductFamily'] = df['ProductFamily'].str.capitalize()

# Setting visual theme
sns.set_theme(style = 'white', palette = None)

# Plot
plt.figure(figsize=(10,7))
sns.countplot(
                data = df,
                y = df['ProductFamily'],
                color='lightsteelblue',
                edgecolor='lightsteelblue',
                order=df.ProductFamily.value_counts().iloc[:10].index
             )
# Layout
plt.suptitle('ProductFamily Frequency', fontsize = 15, fontweight = 'bold')
plt.legend(title='', loc='lower right', prop={'size': 10})
plt.grid()
plt.show()

In [ ]:
# Convert lowercase column using str.lower()
df['ProductDesignation'] = df['ProductDesignation'].str.lower()

# Capitalize first letter
df['ProductDesignation'] = df['ProductDesignation'].str.capitalize()

# Getting sensibility on all 'ProductDesignation'.
Products = list(df.ProductDesignation.unique())
display(Products)

In [ ]:
# Setting visual theme
sns.set_theme(style = 'white', palette = None)

# Plot
plt.figure(figsize=(10,7))
sns.countplot(
                data = df,
                y = df['ProductDesignation'],
                color='lightsteelblue',
                edgecolor='lightsteelblue',
                order=df.ProductDesignation.value_counts().iloc[:10].index
             )
# Layout
plt.suptitle('ProductDesignation', fontsize = 15, fontweight = 'bold')
plt.legend(title='', loc='lower right', prop={'size': 10})
plt.grid()
plt.show()

In [ ]:
# Visualization of the 20 main products ordered by costumers.

plt.rcParams['figure.figsize'] = (10, 10)
wordcloud = WordCloud(background_color = 'white', 
                      width = 1400,  
                      height = 1000, 
                      max_words = 20).generate(str(df['ProductDesignation']))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Most Popular Items bought first by the Customers:',
          fontsize = 20, 
          fontweight = 'bold', 
          x= 0.37, 
          y=1.08)
plt.tight_layout()
plt.show()

In [ ]:
# Under all 'ProductDesignation', understanding what are those that have a 'NO' on the name.
no_list = []
for no_product in Products:
    if ' no ' in no_product:
        product_NO = no_product.strip()
        no_list.append(product_NO)
display(no_list)



# Under all 'ProductDesignation', understanding what are those that have an 'EXTRA' on the name.
extra_list = []
for extra_product in Products:
    if ' extra ' in extra_product:
        product_EXTRA = extra_product.strip()
        extra_list.append(product_EXTRA)
display(extra_list)



# Under all 'ProductDesignation', understanding what are those that have an 'WITH' on the name.
with_list = []
for with_product in Products:
    if ' with ' in with_product:
        product_WITH = with_product.strip()
        with_list.append(product_WITH)
display(with_list)

In [ ]:
# This graph below is very good to obtain sensibility throughout the existing families.
    # It tell us the main preferences of our customers and we can use this information in the end for our market analysis.
# We can see below that in deliveries, there is a tendency for clients to personalize there requests with additional
# products.

# ProductFamily Consumption - Dinne-Inns 
df_dinne_inns = df[df.IsDelivery == 0][["ProductFamily","Qty"]].groupby(["ProductFamily"]).sum().sort_values(by = "Qty", ascending = False).reset_index() 

# ProductFamily Consumption - deliveries (excluding delivery charges)
df_delivery = df[df.ProductDesignation != "DELIVERY CHARGE"][df.IsDelivery == 1][["ProductFamily","Qty"]].groupby(["ProductFamily"]).sum().sort_values(by = "Qty", ascending = False).reset_index() 
df_merged = df_dinne_inns.merge(df_delivery, left_on= "ProductFamily", right_on="ProductFamily" ) 
 
# Plot    
fig = go.Figure(data=[ 
    go.Bar(marker=dict(color='lightsteelblue'),name='Dinne-Inns', x=df_merged['ProductFamily'], y=df_merged['Qty_x']), 
    go.Bar(marker=dict(color='seagreen'),name='Deliveries', x=df_merged['ProductFamily'], y=df_merged['Qty_y']) 
]) 

# Change the bar mode 
fig.update_layout(barmode="group",
                  height=450, 
                  width=950, 
                  title_text="Bestselling products group by families (In Absolute Values)", 
                  xaxis_title="Product Family",
                  yaxis_title="Quantity (Qty)",
                  font=dict(size=16, color='black'),
                  plot_bgcolor="white")
fig.show() 

In [ ]:
# Main conclusions from the graphs below:
    # Spring Roll is the most requested product either for Dinne-Inns as for deliveries. 

df_dinne_inns = df[df.IsDelivery == 0][["ProductDesignation","Qty"]].groupby(["ProductDesignation"]).sum().sort_values(by = "Qty", ascending = False).reset_index() 
df_delivery = df[df.IsDelivery == 1][["ProductDesignation","Qty"]].groupby(["ProductDesignation"]).sum().sort_values(by = "Qty", ascending = False).reset_index() 


# Plot
fig = make_subplots(1,2, subplot_titles=["Dinne-Inns", "Delivery"])
fig.add_trace(go.Bar(marker=dict(color='lightsteelblue'),
                     name='Dinne-Inns', 
                     x=df_dinne_inns['ProductDesignation'][0:20], 
                     y=df_dinne_inns["Qty"][0:20]),1,1)

# We are always excluding delivery charge products
fig.add_trace(go.Bar(marker=dict(color='seagreen'),
                     name='Deliveries', 
                     x=df_delivery[df_delivery['ProductDesignation']!= "DELIVERY CHARGE"]['ProductDesignation'][0:20], 
                     y=df_delivery[df_delivery['ProductDesignation']!= "DELIVERY CHARGE"]["Qty"][0:20]),1,2)
fig.update_layout(height=450, 
                  width=950, 
                  title_text="Bestselling products", 
                  xaxis_title="Product Designation",
                  yaxis_title="Quantity (Qty)",
                  plot_bgcolor="white")
fig.show() 

- **2.6) Verify Data Quality**
- 2.6.6) Checking Qty and Total Amount

In [ ]:
 # Understanding the most required quantities.

df['Qty'].value_counts()/ len(df)

In [ ]:
# 'IsDelivery' distributed per 'Qty' and 'TotalAmount'.
# We can say that the restaurant gains less money with deliveries than with dine-ins.

concat_df_2 = pd.concat([df.groupby(["IsDelivery"])["Qty"].sum(), 
                       df.groupby(["IsDelivery"])["TotalAmount"].sum()], axis=1)
display(concat_df_2)

In [ ]:
# Analyzing the variable 'IsDelivery' graphically.

# Draw
fig, ax = plt.subplots(figsize=(7,6))
g = sns.countplot(data=df, 
                  x=df['IsDelivery'], 
                  color = 'lightsteelblue')
# Decoration
fmt = "{x:,.0f}"
tick = ticker.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)
sns.despine()

# Plot
plt.suptitle('Frequency of IsDelivery', fontsize = 15, fontweight = 'bold')
plt.xlabel("IsDelivery value")
plt.ylabel("Quantity")
plt.rc('axes', labelsize=15)

In [ ]:
# 'CustomerCity' distributed per 'Qty' and 'TotalAmount'.

concat_df_3 = pd.concat([df.groupby(["CustomerCity"])["Qty"].sum(), 
                       df.groupby(["CustomerCity"])["TotalAmount"].sum()], axis=1)
display(concat_df_3)

In [ ]:
# Convert lowercase column using str.lower()
df['CustomerCity'] = df['CustomerCity'].str.lower()

# Capitalize first letter
df['CustomerCity'] = df['CustomerCity'].str.capitalize()

# Setting visual theme
sns.set_theme(style = 'white', 
              palette = None)

# Plot
plt.figure(figsize=(11,8))
sns.countplot(data = df,
              y = df['CustomerCity'],
              color='lightsteelblue',
              edgecolor='lightsteelblue',
              order = df['CustomerCity'].value_counts().index
              )
# Layout
plt.suptitle('CustomerCity', 
             fontsize = 20, 
             fontweight = 'bold')
plt.legend(title='Absolute Values', 
           loc='lower right', 
           prop={'size': 14})
plt.xlabel('Number of Customers', fontsize=14)
plt.ylabel('City/Place/District', fontsize=14)
plt.grid(axis='x')
plt.show()

In [ ]:
# Below, we can have a deeper understanding to answer the following question:
    # "Do more orders correspond to more revenue?"
        # And we can see below, costumer spend, on average, more than the double on restaurant than by deliveries.

# Defining variables.
revenue_dinne_inn = df[df.IsDelivery == 0].TotalAmount.sum()
revenue_delivery = df[df.IsDelivery == 1].TotalAmount.sum()

# Counting.
ordered_dinne_inn = len(df[df.IsDelivery == 0].DocNumber.unique()) # Counting the number of records in dinne-inn.
ordered_delivery = len(df[df.IsDelivery == 1].DocNumber.unique())  # Counting the number of records by delivery.

# Plot.
fig = make_subplots(1,2, 
                    specs=[[{"type":"domain"},
                            {"type":"domain"}]], 
                    subplot_titles=["Revenue", "Number of orders"])
colors = ['lightsteelblue', 'seagreen']
fig.add_trace(go.Pie(labels=["Dinne-Inn", "Deliveries"], 
                     values=[revenue_dinne_inn, revenue_delivery],
                     #textinfo= 'value+percent',
                     marker=dict(colors=colors)),1,1)
fig.add_trace(go.Pie(values=[ordered_dinne_inn, ordered_delivery],
                     marker=dict(colors=colors),
                     showlegend=False),1,2)
fig.update_layout(height=500, 
                  width=650, 
                  title_text="Dinne-Inn vs Deliveries",
                  font=dict(size=12))

# Average Spending per request. 
average_spending_dinneinns = np.mean(df[df.IsDelivery == 0][["DocNumber","TotalAmount"]].groupby("DocNumber").sum()) 
average_spending_deliveries = np.mean(df[df.IsDelivery == 1][["DocNumber","TotalAmount"]].groupby("DocNumber").sum()) 
print("Average Dinne-Inns spending: " + str(round(average_spending_dinneinns[0],2)) + " Euros" "\n" + 
      "Average delivery spending: " + str(round(average_spending_deliveries[0],2))+ " Euros") 
fig.show()

In [ ]:
# Here, we aim to understand how many times normally customers use to visit this resturants chain.
# Most preferences:
    # 5-10 visits: 41%
    # 10+ visits: ~36%
    # 3-5 visits: 22%

on_cust = df[df.IsDelivery == 1]
colors = ["lightsteelblue","seagreen","darkgreen","red","yellow"]
freq = on_cust.CustomerID.value_counts()
freq_bins = pd.Series(["1" if i == 1 else 
                       "2" if i == 2 else 
                       "3-5" if i <= 5 else 
                       "5-10" if i <= 10 else 
                       "10+" for i in freq.values])

# Plot
fig = go.Figure(go.Pie(marker=dict(colors=colors),
                       labels=freq_bins.value_counts().index, 
                       textinfo='percent',
                       values=freq_bins.value_counts().values))
fig.update_layout(height=500, 
                  width=700, 
                  legend_title = "Different number of visits gaps",
                  title_text = "Distribution of the number of visits per client")
fig.show()

- **2.6) Verify Data Quality**
- 2.6.7) Checking EmployeeID

In [ ]:
df['EmployeeID'].value_counts()/ len(df)

In [ ]:
pd.crosstab(df['IsDelivery'], df['EmployeeID'])

In [ ]:
concat_df_3 = pd.concat([df.groupby(["EmployeeID"])["Qty"].sum(), 
                       df.groupby(["EmployeeID"])["TotalAmount"].sum()], axis=1)
display(concat_df_3)

- **2.6) Verify Data Quality**
- 2.6.8) Checking Pax and CustomerID

In [ ]:
# Customer equal to 0 may be what is inserted when the client doesn't want to create a client account in the restaurant.
# Human error, so we will keep it for now.

df['CustomerID'].value_counts()/ len(df)

In [ ]:
# There are some records with pax=0. 
# Since it has a very low representation in the dataset, we consider it a human error. 
# It can perfectly happen that an employee did not take note of the number of costumers of the respective order.

df['Pax'].value_counts()/ len(df)

In [ ]:
# Here, we aim to obtain more sensibility on how many people usually come to the restaurant.
# As we can see, the customers tend to organize themselves to come either in groups of 2 (pairs) 
# or in groups of 4 (prossibility a family).

x = df[(df.IsDelivery == 0) & (df.Pax < 25)].Pax.values

# Plot
fig = px.histogram(x=x)
fig.update_traces(marker_color='lightsteelblue', 
                  opacity=0.7)
fig.update_layout(height=450, 
                  width=950,
                  yaxis_title="Frequency (Absolute Values)", 
                  xaxis_title="Number of persons", 
                  title_text="Number of customers per meal", 
                  showlegend=False, 
                  plot_bgcolor="white")
mean_value = df['Pax'].mean()
fig.add_vline(x=mean_value, 
              line_dash="dash", 
              line_color="seagreen", 
              annotation_text=f"Mean: {mean_value:.2f}")
fig.show()

In [ ]:
# Understanding ou 'TotalAmount' varies with 'Pax' and 'CustomerID'.

df.groupby('TotalAmount')['Pax', 'CustomerID'].sum()

# **3. Data Preparation**

- **3.1) Selecting data**
- 3.1.1) Removing unnecessary rows from Data Exploration
- 3.1.2) Removing unnecessary features from Data Exploration
- 3.1.3) Defining metric and categorical features
- **3.2) Cleaning data**
- 3.2.1) Dealing with missing values and strange values
- 3.2.2) Histograms Analysis
- 3.2.3) Boxplots Analysis
- 3.2.3.1) Manual Removal
- 3.2.3.2) IQR Removal
- 3.2.4) Correlations Analysis
- **3.3) Construct data**
- 3.3.1) Creating new variables

- **3.1) Selecting data**
- 3.1.1) Removing unnecessary rows from Data Exploration

In [ ]:
# Removing this impossible values: clients can not exist before their registration.

df = df[~(df['CustomerSince']>'2018-12-31 23:59:59')]
print('The total amount of impossible values is:', df[df['CustomerSince']>'2018-12-31 23:59:59'].shape[0])
print('Percentage of data kept after removing this impossible values is:', np.round(df.shape[0] / df_original.shape[0], 4))

In [ ]:
# Removing this impossible values: impossible to have an invoice from a client before their registration.

df = df[~(df['InvoiceDateHour'] < df['CustomerSince'])]
print('The total amount of impossible values is:', df[df['InvoiceDateHour'] < df['CustomerSince']].shape[0])
print('Percentage of data kept after removing this impossible values is:', np.round(df.shape[0] / df_original.shape[0], 4))

- **3.1) Selecting data**
- 3.1.2) Removing unnecessary features from Data Exploration

In [ ]:
# RELEVANCY
# Dropping the following three features, which have high cardinality and does not provide any relevant informations: 
    # EmployeeID

df.drop(columns=['EmployeeID'], inplace=True)
df.head(1)

- **3.1) Selecting data**
- 3.1.3) Defining metric and categorical features

In [ ]:
metric_features = df.select_dtypes(include=[np.number])#.columns.tolist()
metric_features = [col for col in metric_features.columns if ('SR' not in col)]
metric_features

In [ ]:
# Defining all CATEGORICAL variables in the dataset. 

categorical_features = [column for column in df.columns if df[column].dtype == 'object']
categorical_features 

- **3.2) Cleaning data**
- 3.2.1) Dealing with missing values and strange values

In [ ]:
# 'CustomerCity' distributed per 'Qty' and 'TotalAmount'.

concat_df_3 = pd.concat([df.groupby(["CustomerCity"])["Qty"].sum(), 
                       df.groupby(["CustomerCity"])["TotalAmount"].sum()], axis=1)
display(concat_df_3)

In [ ]:
# As we noticed above, there are some cities/places/district wrongly written, due to human mistake.
# In this sense, we tried to merge the wrongly spelled ones to the correct ones.


# LAKATAMEIA
# We have confirmed how to spell this name correctly, replacing all those wrongly written citie/splaces/district
# by the correct one. 
# Source:
    # https://www.lakatamia.org.cy/en/home-en/
df['CustomerCity'] = np.where(df['CustomerCity']=='Lakatame',
                              'Lakatameia',df['CustomerCity'])
df['CustomerCity'] = np.where(df['CustomerCity']=='Lakstameia',
                              'Lakatameia',df['CustomerCity'])

# STROBOLOS
# We have confirmed how to spell this name correctly, replacing all those wrongly written cities/places/district 
# by the correct one. 
# Source:
# https://www.strovolos.org.cy/
df['CustomerCity'] = np.where(df['CustomerCity']=='Strobolo',
                              'Strobolos',df['CustomerCity'])


# EGKOMI
# We have confirmed how to spell this name correctly, replacing all those wrongly written cities/places/district
# by the correct one. 
# In this case, we checked that there are two different lines for the same city: EGKOMI.
        # Then, we concluded it could only be due to fact that those values have different datatypes.
# Source:
    # https://ucm.org.cy/en/municipalities/nicosia-district/
    
df['CustomerCity'] = df['CustomerCity'].str.strip()
df['CustomerCity'] = np.where(df['CustomerCity']=='Egkomi',
                              'Egkomi',df['CustomerCity'])

In [ ]:
# After cleaning up our data regarding cities/places, we visualized again our 
# 'CustomerCity' distributed per 'Qty' and 'TotalAmount'.

concat_df_3 = pd.concat([df.groupby(["CustomerCity"])["Qty"].sum(), 
                       df.groupby(["CustomerCity"])["TotalAmount"].sum()], axis=1)
display(concat_df_3)

In [ ]:
# Convert lowercase column using str.lower()
df['CustomerCity'] = df['CustomerCity'].str.lower()

# Capitalize first letter
df['CustomerCity'] = df['CustomerCity'].str.capitalize()

# Setting visual theme
sns.set_theme(style = 'white', 
              palette = None)

# Plot
plt.figure(figsize=(11,8))
sns.countplot(data = df,
              y = df['CustomerCity'],
              color='lightsteelblue',
              edgecolor='lightsteelblue',
              order = df['CustomerCity'].value_counts().index
              )
# Layout
plt.suptitle('CustomerCity', 
             fontsize = 20, 
             fontweight = 'bold')
plt.legend(title='Absolute Values', 
           loc='lower right', 
           prop={'size': 14})
plt.xlabel('Number of Customers', fontsize=14)
plt.ylabel('City/Place/District', fontsize=14)
plt.grid(axis='x')
plt.show()

In [ ]:
# Getting a visual notion of this chain locations restaurants.

# Get the latitude and longitude of the map center
center_lat, center_lon = 35.185566, 33.382275

# Create a map centered on the coordinates above
m = folium.Map(location=[center_lat, center_lon], zoom_start=8)

# Create a Nominatim geolocator
geolocator = Nominatim(user_agent="http")

# Create a rate limiter with a delay of 1 second between requests
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Group the data by city and count the number of unique DocNumbers
grouped = df.groupby('CustomerCity')['DocNumber'].nunique()

# Create a marker for each city and add it to the map
for city, count in grouped.items():
    try:
        # Get the coordinates of the city using OpenStreetMap Nominatim API
        location = geocode(city)
        if location:
            lat, lon = location.latitude, location.longitude
        else:
            # Set the coordinates to (0, 0) if the location is not found
            lat, lon = 0, 0

        # Create a marker and add it to the map
        folium.Marker([lat, lon], popup=f'{city}: {count}').add_to(m)
    except (exc.GeocoderTimedOut, exc.GeocoderServiceError):
        continue

# Display the map
m

- **3.2) Cleaning data**
- 3.2.3) Histograms Analysis

In [ ]:
# HISTOGRAM (METRIC FEATURES)
# Get a depper visual understanding on metric features through histograms.

# Set layout
sns.set_theme(style = 'white', palette = None)
sns.set_style("ticks",{'axes.grid' : False})

n_col = 2
n_row = math.ceil(df[metric_features].shape[1]/n_col)
title = "Metric Variables Histograms"
df[metric_features].hist(bins=10, 
                         figsize=(15, 13), 
                         layout=(n_row, n_col), 
                         xlabelsize=8, ylabelsize=8,
                         color ='lightsteelblue'
                         );

- **3.2) Cleaning data**
- 3.2.4) Boxplots Analysis
- 3.2.4.1) Manual Removal

In [ ]:
# Get a depper visual understanding on METRIC FEATURES through Box Plots. 
# With this graph, we can have an even better understanding on existing outliers.

# All Metric Variables' Box Plots in one figure.
sns.set()
sns.set_theme(style = 'white', 
              palette = None)

# Prepare figure. Create indvidual axes where each box plot will be placed.
fig, axes = plt.subplots(2, 2, figsize=(20, 13))

# Plot data.
# Iterate across axes objects and associate each box plot (hint: use the ax argument):
for ax, feat in zip(axes.flatten(), metric_features): 
    sns.boxplot(x=df[feat], ax=ax, color ='lightsteelblue')
    # Customize
    ax.grid(True)
    
# Layout
title = "Metric Variables Box Plots" # Add a centered title to the figure
fig.suptitle(title, fontsize=16)
plt.show()

In [ ]:
# Outlier Manual Removal (First method).
    # Here, on outliers manual remotion, we decided to look both at the histograms, at the box plots and at the 
    # descriptive statistics resume.
    # In our opinion, all these three were relevant instruments to filter each variable.
        # So, based on these elements, we mostly adopted a 'try and error aproach'.

filters_1 = (
     (df['Pax']<=70) # Mostly, by looking at the boxplots and the histograms.
     &
     (df['Qty']<=(20)) # Mostly, by looking at the boxplots and the histograms.
     &
     (df['TotalAmount']<=(1000)) # Mostly, by looking at the boxplots and the histograms. 
)

df_manual_removal = df[filters_1]

print('Percentage of data kept after removing outliers in df_1:', 
      np.round(df_manual_removal.shape[0] / df_original.shape[0], 4))

- **3.2) Cleaning data**
- 3.2.4) Boxplots Analysis
- 3.2.4.2) IQR Removal

In [ ]:
# Make another copy

df_IQR_removal = df.copy()

In [ ]:
# Source: 

def check_IQR_outliers(data, criterion, mode='data'):
    '''
    Detects outliers by generating a decision range based on a criterion (threshold)
    and locating values that don't fall within this range.
    Can return the dataframe without the outliers or the percentage of remaining data
    after outlier removal or the values considered as outliers for each column by
    changing the parameter 'mode' (data as default)

    Args:
        data (pandas.core.frame.DataFrame) : set of data
        col (pandas.core.series.Series) : column of the data to check outliers
        criterion (int) : IQR multiplier, the higher, the larger the decision range
        mode (str): 'data' to get dataframe without outliers 'perc' to check percentage
                     or 'values' to check values

    Returns:
        Dataframe without
        Print with percentage of remaining data after removing outliers or
        index and values that fall outside the decision range (outliers)
    '''
    if mode == 'data':
        Q1 = data.quantile(.25) # value of first quartile
        Q3 = data.quantile(.75) # value of third quartile
        IQR = Q3 - Q1 # interquartile range
        lower_lim = Q1 - criterion * IQR # setting min limit
        upper_lim = Q3 + criterion * IQR # setting max limit

        outliers = []
        for col in data.select_dtypes(np.number).columns:
            llim = lower_lim[col]
            ulim = upper_lim[col]
            outliers.append(data[col].between(llim, ulim, inclusive='both'))

        # np.all() tests whether all array elements along a given axis evaluate to True
        outliers = data[np.all(outliers, 0)]

        return outliers

    elif mode == 'perc':
        print('\n\033[1mPercentage of remaining data after outlier removal with IQR method:\033[0m\n')
        Q1 = data.quantile(.25) # value of first quartile
        Q3 = data.quantile(.75) # value of third quartile
        IQR = Q3 - Q1 # interquartile range
        lower_lim = Q1 - criterion * IQR # setting min limit
        upper_lim = Q3 + criterion * IQR # setting max limit

        outliers = []
        for col in data.select_dtypes(np.number).columns:
            #outliers.append(data[col].between(lower_lim[col], upper_lim[col], inclusive='both'))
            llim = lower_lim[col]
            ulim = upper_lim[col]
            outliers.append(data[col].between(llim, ulim, inclusive='both'))

        # np.all() tests whether all array elements along a given axis evaluate to True
        outliers = data[np.all(outliers, axis=0)]

        print(f'\033[1m{round(len(outliers)/len(data)*100,3)} %\033[0m of remaining data',
              'after IQR method outlier removal, when criterion is', criterion)

    else:
        print('\n\n\033[1mOutlier Values:\033[0m\n')
        for col in data.select_dtypes(np.number).columns:
            Q1 = data[col].quantile(.25) # value of first quartile
            Q3 = data[col].quantile(.75) # value of third quartile
            IQR = Q3 - Q1 # interquartile range
            lower_lim = Q1 - criterion * IQR # setting min limit
            upper_lim = Q3 + criterion * IQR # setting max limit
            outliers = data.loc[(data[col] > upper_lim) | (data[col] < lower_lim)][col]
            print(f'Outlier values in \033[1m{col}\033[0m:\n')
            for idx, val in outliers.items():
                print(f'Value from Client {idx}: \033[1m', round(val,2),'\033[0m\n')

In [ ]:
# Checking percentage of remaining data
check_IQR_outliers(data=df_IQR_removal, criterion=3, mode='perc')

print('Percentage of data kept after removing outliers in df_1:', 
      np.round(df_IQR_removal.shape[0] / df_original.shape[0], 4))

**Key Takeawyays**
- As we verified above, the IQR method, removes any value at all from our dataset.
- Since under the manual approach, we have more accuracy on the data we are removing, we opted for that approach.

- **3.2) Cleaning data**
- 3.2.5) Correlations Analysis

In [ ]:
# The only two variables that are highly correlated are "Pax" and "IsDelivery".
# However, as long as they are key for our analysis, the decided to do not drop any of them.

def cor_heatmap(cor):
    plt.figure(figsize=(20,20))
    sns.heatmap(data = cor, 
                annot = True, 
                cmap = 'PuBu', 
                fmt='.1')
    plt.title('Final Dataset Spearman Correlation Heatmap',fontsize = 34, fontweight = 'bold')
    plt.show()
    
# Apply the correlation method to my dataset, usins spearman.
cor_spearman = df[metric_features].corr(method= 'spearman')
cor_heatmap(cor_spearman)

- **3.3) Construct data**
- 3.3.1) Creating new variables

In [ ]:
# Creating 'day' variable.
df['day'] = df['InvoiceDateHour'].apply(lambda x:x.day)

# Creating 'month' variable.
df['month'] = df['InvoiceDateHour'].apply(lambda x:x.month)

# Creating 'year' variable.
df['year'] = df['InvoiceDateHour'].apply(lambda x:x.year)

# Creating 'Hour_of_day' variable.
df['Hour_of_day'] = df['InvoiceDateHour'].apply(lambda x:x.hour)

In [ ]:
# Drawing a graph for 'Invoice's Day' variable sorted descendently.

# Setting visual theme
sns.set_theme(style = 'white', 
              palette = None)

# Plot
plt.figure(figsize=(10,7))
sns.countplot(data = df,
              x = df['day'],
              color='lightsteelblue',
              edgecolor='lightsteelblue',
              order = df['day'].value_counts().index
              )

# Layout
plt.suptitle('Countplot for Days', 
             fontsize = 15, 
             fontweight = 'bold')
plt.legend(title='', loc='lower right', prop={'size': 10})
plt.xlabel('Days')
plt.ylabel('Frequency (In Absolute Value)')
plt.grid()
plt.show()

In [ ]:
# Drawing a graph for 'Invoice's Month' variable sorted descendently.
# Setting visual theme
sns.set_theme(style = 'white', palette = None)

# Plot
plt.figure(figsize=(10,7))
sns.countplot(data = df,
              y = df['month'],
              color='lightsteelblue',
              edgecolor='lightsteelblue',
              order = df['month'].value_counts().index
              )

# Layout
plt.suptitle('Countplot for Month', fontsize = 15, fontweight = 'bold')
plt.legend(title='', loc='lower right', prop={'size': 10})
plt.xlabel('Frequency (In Absolute Value)')
plt.ylabel('Month')
plt.grid()
plt.show()

In [ ]:
# Drawing a graph for 'Invoice's Hour_of_day' variable sorted descendently.
# Setting visual theme
sns.set_theme(style = 'white', palette = None)

# Plot
plt.figure(figsize=(10,7))
sns.countplot(data = df,
              x = df['Hour_of_day'],
              color='lightsteelblue',
              edgecolor='lightsteelblue',
              order = df['Hour_of_day'].value_counts().index
              )

# Layout
plt.suptitle('Countplot for Hour of the day', fontsize = 15, fontweight = 'bold')
plt.legend(title='', loc='lower right', prop={'size': 10})
plt.xlabel('Hour of the day')
plt.ylabel('Frequency (In Absolute Value)')
plt.grid()
plt.show()

In [ ]:
# Understanding on which day of the week were the invoices issued.
# Creating Invoice's Day of the Week attribute
df['date_name'] = df['InvoiceDateHour'].dt.day_name()

# # Visualizing the output
# Setting visual theme
sns.set_theme(style = 'white', palette = None)

# Plot
plt.figure(figsize=(10,7))
sns.countplot(data = df,
              y = df['date_name'],
              color='lightsteelblue',
              edgecolor='lightsteelblue',
              order = df['date_name'].value_counts().index
              )
# Layout
plt.suptitle('Countplot for Day of the Week', fontsize = 15, fontweight = 'bold')
plt.legend(title='', loc='lower right', prop={'size': 10})
plt.xlabel('Frequency (In Absolute Value)')
plt.ylabel('Date of the Week')
plt.grid()
plt.show()

In [ ]:
#df['InvoiceDateHour_datetime'] = pd.to_datetime(df['InvoiceDateHour'], format='%Y-%m-%d %H:%M:%S.%f')
#df.drop(columns=['InvoiceDateHour'],inplace=True)


# Understanding on which day of the week were the invoices issued.
# Understanding whether the invoice was issued on weekend or not. Its a binary variable.
# Understanding what what time of the day the invoice was issued.
# Visualizing the output.

# Source:
    # https://stackoverflow.com/questions/62884585/select-weekends-and-weekdays-by-weekday-in-python

df['Weekday_Invoice'] = df.InvoiceDateHour.dt.dayofweek
df['Weekend_Invoice'] = np.where(df.InvoiceDateHour.dt.dayofweek.isin([5,6]), 1, 0)
df['Hour_of_day'] = df['InvoiceDateHour'].dt.hour

# Storing variables in a new dataframe called 'weekends'.
weekends = df[df['Weekend_Invoice']==1]

# Storing variables in a new dataframe called 'workdays'.
workdays = df[df['Weekend_Invoice']!=1]
display(weekends)

In [ ]:
# Getting a graphic visualization of the sales distribution by day of the week and by hour.

# Conclusions:
    # Costumers have similar conclusions throughout the week.
    # Sunday shows more costumers during lunch time and less picks during dinner time.

# Draw
fig, ax = plt.subplots(figsize=(10,7))
g = joypy.joyplot(data=df, 
                  column='Hour_of_day', 
                  by='date_name',
                  ax=ax,
                  color='lightsteelblue') # set color parameter to lightsteelblue

# Plot
plt.title("Distribution by weekday and Hour_of_day", 
          fontsize=15, 
          fontweight = 'bold')
plt.xlabel("Hour_of_day")
plt.rc('axes', 
       labelsize=12)

In [ ]:
# Totals sales by day of the week and delivery

# Pivot table
aggregated_df = df.pivot_table(values=['TotalAmount'], 
                               index='date_name',
                               columns='IsDelivery',
                               aggfunc='sum',
                               fill_value=0)

# Draw
fig, ax = plt.subplots(figsize=(10,7))
g = aggregated_df.plot(kind='bar', 
                       stacked=True, 
                       ax=ax, 
                       color=['lightsteelblue', 'seagreen'])

# Decoration
fmt = "{x:,.0f}"
tick = ticker.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)
sns.despine()
handles, labels = ax.get_legend_handles_labels()
labels = ['Dinne - Inn Situation', 'Delivery Situation']  # updated labels
ax.legend(handles=handles, 
          labels=labels, 
          loc='upper center', 
          ncol=4, 
          bbox_to_anchor=(0.5, 1.2),  # updated position
          frameon=False)

# Plot
plt.title("Sales by weekday and type of sell", fontsize=15, fontweight='bold')
plt.xlabel("Weekday (including Weekends)")
plt.ylabel("Total sales amount")
plt.xticks(rotation=0, ha='center')
plt.rc('axes', labelsize=12)

In [ ]:
# We aimed to understand at which time of the week and at which time of the day, people use to come
# to the restaurants.

# Conclusions:
    # The most busy days of the week are:
        # Friday
        # Saturday
    # Sunday presents usually has some busy lunch hours. 
        

grouped_df = df.groupby(["date_name", "Hour_of_day"])["TotalAmount"].aggregate("count").reset_index()
grouped_df = grouped_df.pivot('date_name', 'Hour_of_day', 'TotalAmount')

plt.figure(figsize=(15,8))
sns.heatmap(grouped_df, cmap='PuBu')
plt.title("Frequency of Day of week Vs Hour of the day", fontsize=15, fontweight='bold')
plt.show()

In [ ]:
# Creating Invoice's Day of the Week attribute
'''
df.loc[(df['month']>= 1) &(df['month'] <=2), 'season'] = "Winter"
df.loc[(df['month']>= 3) &(df['month'] <=4), 'season'] = "Spring"
df.loc[(df['month']>= 5) &(df['month'] <=6), 'season'] = "Summer"
df.loc[(df['month']>= 7) &(df['month'] <=8), 'season'] = "Monsoon"
df.loc[(df['month']>= 9) &(df['month'] <=10), 'season'] = "Autumn"
df.loc[(df['month']>= 11) &(df['month'] <=12), 'season'] = "Winter"
'''

df.loc[(df['month']>= 1) &(df['month'] <=2), 'season'] = "Winter"
df.loc[(df['month']>= 3) &(df['month'] <=5), 'season'] = "Spring"
df.loc[(df['month']>= 6) &(df['month'] <=8), 'season'] = "Summer"
df.loc[(df['month']>= 9) &(df['month'] <=11), 'season'] = "Autumn"
df.loc[(df['month']> 11) &(df['month'] <=12), 'season'] = "Winter"

# Drawing a graph for Invoice's Day of the Week attribute
# Setting visual theme
sns.set_theme(style = 'white', palette = None)

# Plot
plt.figure(figsize=(10,7))

sns.countplot(data = df,
              y = df['season'],
              color='lightsteelblue',
              edgecolor='lightsteelblue',
              order = df['season'].value_counts().index
              )
# Layout
plt.suptitle('Countplot for Seasons', fontsize = 15, fontweight = 'bold')
plt.legend(title='', loc='lower right', prop={'size': 10})
plt.xlabel("Frequency")
plt.ylabel("Season")
plt.grid()
plt.show()

In [ ]:
# Drawing a graph for Total Amount per Day
# Setting visual theme
sns.set_theme(style = 'white', palette = None)

# Plot
plt.figure(figsize=(10,7))

sns.lineplot(x="day", 
             y="TotalAmount",
             hue="IsDelivery",
             palette = ["#aec7e8", '#2ca02c'], 
             data=df)
# set the title and labels
plt.suptitle('TotalAmount per day - Delivery x Restaurant', fontsize=15, fontweight='bold', y=1.05)
plt.xlabel('Day')
plt.ylabel('Total Amount')
#plt.legend(['Restaurant', 'Delivery'], loc='lower right', prop={'size': 10})
plt.grid()

# rotate and move the y-axis label
plt.ylabel('Total Amount', ha='right',fontsize=16, labelpad = 20)
ax = plt.gca()
ax.yaxis.set_label_coords(-0.05, 0.5)

#plt.title('Frequency of Customers by City', fontsize=20, fontweight='bold')
plt.xlabel('Per Day', fontsize=16)
plt.ylabel('Total Amount', fontsize=16)

# display the plot
plt.show()

In [ ]:
# Creating National Holidays attribute
# Based on https://www.timeanddate.com/holidays/cyprus/2018

df['Date'] = df.InvoiceDateHour.apply(lambda x: x.date())

NationalHolidays = ['01-01-2018','06-01-2018','19-02-2018','25-03-2018','01-04-2018',
                    '06-04-2018','07-04-2018', '08-04-2018','09-04-2018','01-05-2018',
                    '28-05-2018','15-08-2018','01-10-2018','28-10-2018', '24-12-2018',
                    '25-12-2018','26-12-2018','31-12-2018']

holidays_date = pd.to_datetime(NationalHolidays, format='%d-%m-%Y')
holiday_dates = holidays_date.strftime('%Y-%m-%d').tolist()

df['Holiday'] = df.Date.apply(lambda x: 1 if str(x) in holiday_dates else 0)

display(df.head())

In [ ]:
# Drawing a graph for Invoice's Day of the Week attribute
# Setting visual theme
sns.set_theme(style = 'white', palette = None)

# Plot
plt.figure(figsize=(7,5))

sns.countplot(data = df,
              y = df['Holiday'],
              color='lightsteelblue',
              edgecolor='lightsteelblue',
              order = df['Holiday'].value_counts().index
              )
# Layout
plt.suptitle('Countplot for Holidays', fontsize = 15, fontweight = 'bold')
plt.legend(title='', loc='lower right', prop={'size': 10})
plt.xlabel('Frequency (In Absolute Values)')
plt.ylabel('Holiday')
plt.grid()
plt.show()

In [ ]:
# The 'three most profitable' holidays are:
    # 24.12.2018, Christmas Eve
    # 31.12.2018, New Year's Eve
    # 06.01.2018, Epiphany
# This three holidays are separated for just 13 days. We suggest a special campaign during these two weeks. 

# Three 'two less profitable' holidays are:
    # 01.05.2018, Labour Day/May Day
    # 28.05.2018, Orthodox Pentecost Monday

# Group the data by holiday status and date and calculate the total revenue for each group
revenue_by_holiday = df[df['Holiday'] == 1].groupby(['Holiday', 'Date'])['TotalAmount'].sum()

# Layout
# Plot
ax = revenue_by_holiday.plot.bar(figsize=(10, 8),
                                color = ['lightsteelblue'])
ax.set_title('Total Revenue by Holiday')
ax.set_xlabel('Holiday')
ax.set_ylabel('TotalAmount')
plt
plt.show()

# **4. Modeling**


- **4.1) Association Rules**
- 4.1.1) Creating Pivot tables for apriori
- **4.2) Modelling: By Product**
- 4.2.2) Overall DataSet: Defining and Exploring some metrics as support, confidence and lift.
- 4.2.2) Key Takeaways
- **4.3) Modelling: Dinne-Inns**
- 4.3.1) Overall DataSet: Defining and Exploring some metrics as support, confidence and lift.
- 4.3.2) Key Takeaways
- **4.4) Modelling: Deliveries**
- 4.4.1) Overall DataSet: Defining and Exploring some metrics as support, confidence and lift.
- 4.4.2) Key Takeaways
- **4.5) Modelling: Dinne-Inns Excluding Water**
- 4.5.1) Overall DataSet: Defining and Exploring some metrics as support, confidence and lift.
- 4.5.2) Key Takeaways
- **4.6) Modelling: By Family**
- 4.6.1) Overall DataSet: Defining and Exploring some metrics as support, confidence and lift.
- 4.6.2) Key Takeaways
- **4.7) Menus Suggestions**
- 4.7.1) Menus for Dinne-Inns
- 4.7.2) Menus for Deliveries

- **4.1) Association Rules**
- 4.1.1) Creating Pivot tables for apriori

- **ASSUMPTIONS**
<br>
<br>
- **First Approach:** Firstly, we tried to understand the main conclusions analyzing both by product and by product family, without including any filter in our selection.
<br>
- **Second Approach:** As we saw previously throughout our report, since there are very different behaviours patterns between Dinne-Inns and Deliveries situations, we decided to create two different datasets and analyzing them separately. Additionally, in the Deliveries dataset, we decided to not include two products: Delivery Charge and Tsanta, both products related with the delivery process (not a meal itself), so that we get a cleaner visualization of the association rules.
<br>
- **Third Approach (Dinne - Inns):** after the first iteration, we noticed that the product Mineral water 1.5lt appeared very frequently in the association rules. In this sense, we decided to create a new dataset, derived from dinne-inn, but without including Mineral water 1.5lt, in order to understand deeply the behaviour patterns of the costumers.

In [ ]:
# Here, we are creating the pivot tables to apply the apriori.

# 1. By Product, no restriction
df_pt = pd.pivot_table(df[['DocNumber','ProductDesignation']],
                    index='DocNumber',                                 # Each line represents a document (invoice).
                    columns='ProductDesignation',                      # Each column will represents a feature.
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)  # The value should be 1 if the product is present in the document, otherwise 0.

# 2. By Product, delivery.
df_pt_delivery = pd.pivot_table(df[(df.IsDelivery == 1)&(df.ProductDesignation != 'Tsanta')][["DocNumber","ProductDesignation"]], 
                         index='DocNumber', 
                         columns='ProductDesignation', 
                         aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0).drop("Delivery charge",axis=1) 


# 3. By Product, dinne_inn.
df_pt_dinne_in = pd.pivot_table(df[df.IsDelivery == 0][["DocNumber","ProductDesignation"]], 
                                index='DocNumber', 
                                columns='ProductDesignation', 
                                aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0) 


# 4. Dinne-Inns - excluding Water, since its highly unbalanced.
df_pt_dinne_in_no_water = pd.pivot_table(df[(df.IsDelivery == 0)&(df.ProductDesignation != 'Mineral water 1.5lt')][["DocNumber","ProductDesignation"]], 
                            index='DocNumber', 
                            columns='ProductDesignation', 
                            aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0) 


# 5. By Family, no restriction.
df_pt_family = pd.pivot_table(df[['DocNumber','ProductFamily']],
                    index='DocNumber',                                  # Each line represents a document (invoice).
                    columns='ProductFamily',                            # Each column will represents a feature.
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)   # The value should be 1 if the product is present in the document, otherwise 0


# Visualizing the table 1, by product with no restriction.
print("If we do not define any threshold, we have the following amount of association rules:", df_pt.shape)
df_pt.head()

- **4.2) Modelling: Overall DataSet**
- 4.2.2) Overall DataSet: Defining and Exploring some metrics as support, confidence and lift.

- **Itemset:** "list of all the items in the antecedent and the consequent".
- **Support:** "the fraction of the total number of transactions in which the itemset occurs (%)". 
- **Antecedent Support:** represents the frequency of the antecedent itemset in the dataset (%).
- **Consequent Support:** represents the frequency of the consequent itemset in the dataset (%).
- **Confidence:** "the conditional probability of occurrence of consequent given the antecedent (%)".
- **Lift:** "the conditional probability of occurrence of consequent given the antecedent".

In [ ]:
# Here, we aim to apply the APRIORI algorithm.
# The 'min_support' parameter, measn that this rules are supported in more than 5% of the transactions.
# Source:
    # http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

# We can define the itemset as "the list of all the items in the antecedent and the consequent".
# Source:
    # https://towardsdatascience.com/association-rules-2-aa9a77241654
    # https://www.geeksforgeeks.org/association-rule/

frequent_itemsets = apriori(df_pt, min_support=0.05, use_colnames=True)

**Defining a threshold of 10% for support** <br>
**Defining a threshold of 50% for confidence** <br>
**Defining a threshold of 1.5 for lift** <br>

In [ ]:
# Here, we aim to generate the association rules - by support.
# According to the same source, the support is "the fraction of the total number of transactions in which the itemset occurs".
# Source:
    # https://towardsdatascience.com/association-rules-2-aa9a77241654
    # https://www.geeksforgeeks.org/association-rule/
    # https://www.ibm.com/docs/en/db2/9.7?topic=associations-support-in-association-rule
rules_support = association_rules(frequent_itemsets, metric="support", min_threshold=0.1)
rules_support.sort_values(by='support', ascending=False, inplace=True)



# Here, we aim to generate the association rules - by confidence.
# According to the same source, the confidence is "the conditional probability of occurrence of 
# consequent given the antecedent".
# Source:
    # https://towardsdatascience.com/association-rules-2-aa9a77241654
    # https://www.geeksforgeeks.org/association-rule/
rules_confidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.50)
rules_confidence.sort_values(by='confidence', ascending=False, inplace=True)



# Here, we aim to generate the association rules - by lift.
# According to the same source, the lift is "the conditional probability of occurrence of 
# consequent given the antecedent".
# Source:
    # https://towardsdatascience.com/association-rules-2-aa9a77241654
    # https://www.geeksforgeeks.org/association-rule/
    # https://www.ibm.com/docs/en/db2/11.1?topic=SSEPGG_11.1.0/com.ibm.im.model.doc/c_lift_in_an_association_rule.htm
rules_lift = association_rules(frequent_itemsets, metric="lift", min_threshold=1.5)
rules_lift.sort_values(by='lift', ascending=False, inplace=True)

# Visualizing the lift values sorted descendently.
rules_lift.sort_values(by=['lift'], ascending=False).head(5)

In [ ]:
# Scatter plot of these rules.
# We cand understand below that normally, there is a tendency for a positive correlation between
# the metrics 'support' and 'confidence', although without a very high value in absolute terms.
# The size


# Draw
fig, ax = plt.subplots(figsize=(10,7))
g = sns.scatterplot(data=rules_support, 
                    x="support", 
                    y="confidence", 
                    size="lift", 
                    sizes=(30, 250), 
                    alpha=0.8)

# Plot
sns.despine()
plt.title("Rules with a minimum support threshold of 10% (Lift represeted by the size)", fontsize=12)
plt.xlabel("Support level")
plt.ylabel("Confidence level")
plt.rc('axes', labelsize=12)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, loc='upper center', 
          ncol=6, bbox_to_anchor=(0.45, 1.03), frameon=False)

In [ ]:
# Bubble plot of the rules.

# The plot displays the relationship between the support and confidence levels of different rules in a dataset, 
# where each point represents a rule. 
# In this plot, the size of each point (ball) represents the lift, which is a measure of the 
# strength of the association between the antecedent and consequent of a rule. 
# Points with a higher lift will have a bigger size, and points with a lower lift will have a smaller size.
    # Therefore, bigger balls on the plot indicate that the corresponding rules have a higher lift, 
    # which means a stronger association between the antecedent and consequent of the rule.

# Replace frozen sets with strings
rules_confidence['antecedents_'] = rules_confidence['antecedents'].apply(lambda a: ','.join(list(a)))
rules_confidence['consequents_'] = rules_confidence['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(14,14))
ax= plt.scatter(data=rules_confidence, 
                x='consequents_', 
                y='antecedents_', 
                s = rules_confidence['lift']*20, 
                edgecolors = "blue", 
                c = "white", 
                zorder = 2)

# Plot
nRules=rules_confidence.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules", fontsize=12)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls = "--", zorder = 1)
fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
# General summary for rules lift threshold defined.

rules_lift.describe()

**Exploring the most frequent items**

In [ ]:
# Measuring the quantity of products per set.

frequent_itemsets['products_per_set'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))


# Checking the results for a 'products_per_set' = 2 and a Support >= 0.1.
# Reminding, a support repreents the percentage of transactions in the entire dataset that contain a particular itemset. 
# In this case, we aim to filter all the itemsets that represents at least 10% of the total transactions.

frequent_itemsets[(frequent_itemsets['products_per_set'] == 2) & (frequent_itemsets['support'] >= 0.17)]

In [ ]:
# Itemsets with both high 'Confidence' and high 'Lift'.

rules_confidence[(rules_confidence['lift'] >= 3) & (rules_confidence['confidence'] >= 0.7)]

In [ ]:
# First draft.
# Plot a basic network graph of the top 20 confidence rules

# Create a copy of the rules and transform the frozensets to strings
rulesToPlot = rules_confidence.copy(deep=True)
rulesToPlot['LHS'] = [','.join(list(x)) for x in rulesToPlot['antecedents']]
rulesToPlot['RHS'] = [','.join(list(x)) for x in rulesToPlot['consequents']]

# Remove duplicate if reversed rules
rulesToPlot['sortedRow'] = [sorted([a,b]) for a,b in zip(rulesToPlot.LHS, rulesToPlot.RHS)]
rulesToPlot['sortedRow'] = rulesToPlot['sortedRow'].astype(str)
rulesToPlot.drop_duplicates(subset=['sortedRow'], inplace=True)

# Plot Graphically
rulesToPlot=rulesToPlot[:20]
fig = plt.figure(figsize=(25, 25)) 
G = nx.from_pandas_edgelist(rulesToPlot, 'LHS', 'RHS')
np.random.seed(1000)
nx.draw(G, with_labels=True, 
        node_size=30, 
        node_color="lightsteelblue", 
        font_size = 18,
        pos=nx.spring_layout(G))
plt.axis('equal')
plt.show()

- **4.2) Modelling: By Product**
- 4.2.2) Key Takeaways <br>
<br>
- **Most frequent consequents:** Egg Fried Rice, Mineral Water 1.5LT, Delivery Charge and Noodles with meat.
- All the dots that are closer to each of these "centers", are the most usual antecedents.
- The closer is the point, the higher is the confidence level.
- We still verify that the product Tsanta is closely linked with Delivery Charge. Both most not be taken into consideration. 

- **4.3) Modelling: Dinne-Inns**
- 4.3.1) Overall DataSet: Defining and Exploring some metrics as support, confidence and lift.

**Defining a threshold of 10% for support** <br>
**Defining a threshold of 50% for confidence** <br>
**Defining a threshold of 0 for lift (to analyze possible substitute products)** <br>

In [ ]:
# DINNE INNS
frequent_itemsets_dinne_in = apriori(df_pt_dinne_in, min_support=0.05, use_colnames=True) 

# Defining Support
rules_support_dinne_in = association_rules(frequent_itemsets_dinne_in, metric="support", min_threshold=0.10) 
rules_support_dinne_in.sort_values(by='support', ascending=False, inplace=True) 
print(rules_support_dinne_in.head(3))

# Defining Confidence
rules_confidence_dinne_in = association_rules(frequent_itemsets_dinne_in, metric="confidence", min_threshold=0.50) 
rules_confidence_dinne_in.sort_values(by='confidence', ascending=False, inplace=True) 
print(rules_confidence_dinne_in.head(3))
 
# Defining Lift
rules_lift_dinne_in = association_rules(frequent_itemsets_dinne_in, metric="lift", min_threshold=0)  
rules_lift_dinne_in.sort_values(by='lift', ascending=False, inplace=True) 
rules_lift_dinne_in.head(3)

In [ ]:
# First draft.
# Plot a basic network graph of the top 20 confidence rules

# Create a copy of the rules and transform the frozensets to strings
rulesToPlot_dinne_in = rules_confidence_dinne_in.copy(deep=True)
rulesToPlot_dinne_in['LHS'] = [','.join(list(x)) for x in rulesToPlot_dinne_in['antecedents']]
rulesToPlot_dinne_in['RHS'] = [','.join(list(x)) for x in rulesToPlot_dinne_in['consequents']]

# Remove duplicate if reversed rules
rulesToPlot_dinne_in['sortedRow'] = [sorted([a,b]) for a,b in zip(rulesToPlot_dinne_in.LHS, rulesToPlot_dinne_in.RHS)]
rulesToPlot_dinne_in['sortedRow'] = rulesToPlot_dinne_in['sortedRow'].astype(str)
rulesToPlot_dinne_in.drop_duplicates(subset=['sortedRow'], inplace=True)

# Plot Graphically
rulesToPlot_dinne_in=rulesToPlot_dinne_in[:20]
fig = plt.figure(figsize=(22, 22)) 
G = nx.from_pandas_edgelist(rulesToPlot_dinne_in, 'LHS', 'RHS')
np.random.seed(1000)
nx.draw(G, with_labels=True, 
        node_size=30, 
        node_color="red", 
        font_size = 18,
        pos=nx.spring_layout(G))
plt.axis('equal')
plt.show()

- **4.3) Modelling: Dinne-Inns**
- 4.3.2) Key Takeaways <br>
<br>
- **Most frequent consequents:** Mineral Water 1.5LT and Noodles with meat.
- All the dots that are closer to each of these "centers", are the most usual antecedents.
- The closer is the point, the higher is the confidence level.

- **4.4) Modelling: Deliveries**
- 4.4.1) Overall DataSet: Defining and Exploring some metrics as support, confidence and lift.

**Defining a threshold of 10% for support** <br>
**Defining a threshold of 50% for confidence** <br>
**Defining a threshold of 0 for lift (to analyze possible substitute products)** <br>

In [ ]:
# DElIVERIES
frequent_itemsets_delivery = apriori(df_pt_delivery, min_support=0.05, use_colnames=True) 

# Defining Support
rules_support_delivery = association_rules(frequent_itemsets_delivery, metric="support", min_threshold=0.10) 
rules_support_delivery.sort_values(by='support', ascending=False, inplace=True) 
print(rules_support_delivery.head(3))

# Defining Confidence
rules_confidence_delivery = association_rules(frequent_itemsets_delivery, metric="confidence", min_threshold=0.50) 
rules_confidence_delivery.sort_values(by='confidence', ascending=False, inplace=True) 
print(rules_confidence_delivery.head(3))
 
# Defining Lift
rules_lift_delivery = association_rules(frequent_itemsets_delivery, metric="lift", min_threshold=0)  
rules_lift_delivery.sort_values(by='lift', ascending=False, inplace=True) 
rules_lift_delivery.head(3) 

In [ ]:
# First draft.
# Plot a basic network graph of the top 20 confidence rules

# Create a copy of the rules and transform the frozensets to strings
rulesToPlot_delivery = rules_confidence_delivery.copy(deep=True)
rulesToPlot_delivery['LHS'] = [','.join(list(x)) for x in rulesToPlot_delivery['antecedents']]
rulesToPlot_delivery['RHS'] = [','.join(list(x)) for x in rulesToPlot_delivery['consequents']]

# Remove duplicate if reversed rules
rulesToPlot_delivery['sortedRow'] = [sorted([a,b]) for a,b in zip(rulesToPlot_delivery.LHS, rulesToPlot_delivery.RHS)]
rulesToPlot_delivery['sortedRow'] = rulesToPlot_delivery['sortedRow'].astype(str)
rulesToPlot_delivery.drop_duplicates(subset=['sortedRow'], inplace=True)

# Plot Graphically
rulesToPlot_delivery=rulesToPlot_delivery[:15]
fig = plt.figure(figsize=(17, 17)) 
G = nx.from_pandas_edgelist(rulesToPlot_delivery, 'LHS', 'RHS')
np.random.seed(1000)
nx.draw(G, with_labels=True, 
        node_size=30, 
        node_color="red", 
        font_size = 18,
        pos=nx.spring_layout(G))
plt.axis('equal')
plt.show()

- **4.4) Modelling: Deliveries**
- 4.4.2) Key Takeaways <br>
<br>
- **Most frequent consequents:** Sweet Sour Chicken and Egg Fried Rice.
- All the dots that are closer to each of these "centers", are the most usual antecedents.
- The closer is the point, the higher is the confidence level.
- There are three smaller groups, not linked with the main ones above referred which means that this products are not usually ordered along with the remaning clusters of products.

- **4.5) Modelling: Dinne-Inns Excluding Water**
- 4.5.1) Overall DataSet: Defining and Exploring some metrics as support, confidence and lift.

**Defining a threshold of 10% for support** <br>
**Defining a threshold of 50% for confidence** <br>
**Defining a threshold of 0 for lift (to analyze possible substitute products)** <br>

In [ ]:
# DINNE INNS EXCLUDING WATER
frequent_itemsets_dinne_in_no_water = apriori(df_pt_dinne_in_no_water, min_support=0.05, use_colnames=True) 

# Defining Support
rules_support_dinne_in_no_water = association_rules(frequent_itemsets_dinne_in_no_water, metric="support", min_threshold=0.10) 
rules_support_dinne_in_no_water.sort_values(by='support', ascending=False, inplace=True) 
print(rules_support_dinne_in_no_water.head(3))

# Defining Confidence
rules_confidence_dinne_in_no_water = association_rules(frequent_itemsets_dinne_in_no_water, metric="confidence", min_threshold=0.50) 
rules_confidence_dinne_in_no_water.sort_values(by='confidence', ascending=False, inplace=True) 
print(rules_confidence_dinne_in_no_water.head(3))
 
# Defining Lift
rules_lift_dinne_in_no_water = association_rules(frequent_itemsets_dinne_in_no_water, metric="lift", min_threshold=0)  
rules_lift_dinne_in_no_water.sort_values(by='lift', ascending=False, inplace=True) 
rules_lift_dinne_in_no_water.head(3) 

In [ ]:
# First draft.
# Plot a basic network graph of the top 15 confidence rules

# Create a copy of the rules and transform the frozensets to strings
rulesToPlot_dinne_in_no_water = rules_confidence_dinne_in_no_water.copy(deep=True)
rulesToPlot_dinne_in_no_water['LHS'] = [','.join(list(x)) for x in rulesToPlot_dinne_in_no_water['antecedents']]
rulesToPlot_dinne_in_no_water['RHS'] = [','.join(list(x)) for x in rulesToPlot_dinne_in_no_water['consequents']]

# Remove duplicate if reversed rules
rulesToPlot_dinne_in_no_water['sortedRow'] = [sorted([a,b]) for a,b in zip(rulesToPlot_dinne_in_no_water.LHS, rulesToPlot_dinne_in_no_water.RHS)]
rulesToPlot_dinne_in_no_water['sortedRow'] = rulesToPlot_dinne_in_no_water['sortedRow'].astype(str)
rulesToPlot_dinne_in_no_water.drop_duplicates(subset=['sortedRow'], inplace=True)

# Plot Graphically
rulesToPlot_dinne_in_no_water=rulesToPlot_dinne_in_no_water[:15]
fig = plt.figure(figsize=(17, 17)) 
G = nx.from_pandas_edgelist(rulesToPlot_dinne_in_no_water, 'LHS', 'RHS')
np.random.seed(1000)
nx.draw(G, with_labels=True, 
        node_size=30, 
        node_color="red", 
        pos=nx.spring_layout(G))
plt.axis('equal')
plt.show()

- **4.5) Modelling: Dinne-Inns Excluding Water**
- 4.5.2) Key Takeaways<br>
<br>
- **Most frequent consequents:** Egg Fried Rice, Spring Roll and No Meat.
- All the dots that are closer to each of these "centers", are the most usual antecedents.
- The closer is the point, the higher is the confidence level.
- We can also observe that Toffe Banana Complementary and Beef BBS are an antececedent to both the consequence above mentioned.
- For example when compared with the dataset **WITH WATER**, we can understand here the relevance of Egg fried rice.

- **4.6) Modelling: By Family**
- 4.6.1) Overall DataSet: Defining and Exploring some metrics as support, confidence and lift.

**Defining a threshold of 20% for support (since the granularity is bigger, the support is also higher, so we also increased the support threshold)** <br>
**Defining a threshold of 50% for confidence** <br>
**Defining a threshold of 0 for lift (to analyze possible substitute products)** <br>

In [ ]:
# DINNE INNS EXCLUDING WATER
frequent_itemsets_family = apriori(df_pt_family, min_support=0.05, use_colnames=True) 

# Defining Support
rules_support_family = association_rules(frequent_itemsets_family, metric="support", min_threshold=0.20) 
rules_support_family.sort_values(by='support', ascending=False, inplace=True) 
print(rules_support_family.head(3))

# Defining Confidence
rules_confidence_family = association_rules(frequent_itemsets_family, metric="confidence", min_threshold=0.50) 
rules_confidence_family.sort_values(by='confidence', ascending=False, inplace=True) 
print(rules_confidence_family.head(3))
 
# Defining Lift
rules_lift_family = association_rules(frequent_itemsets_family, metric="lift", min_threshold=0)  
rules_lift_family.sort_values(by='lift', ascending=False, inplace=True) 
rules_lift_family.head(5) 

In [ ]:
# First draft.
# Plot a basic network graph of the top 15 confidence rules

# Create a copy of the rules and transform the frozensets to strings
rulesToPlot_family = rules_confidence_family.copy(deep=True)
rulesToPlot_family['LHS'] = [','.join(list(x)) for x in rulesToPlot_family['antecedents']]
rulesToPlot_family['RHS'] = [','.join(list(x)) for x in rulesToPlot_family['consequents']]

# Remove duplicate if reversed rules
rulesToPlot_family['sortedRow'] = [sorted([a,b]) for a,b in zip(rulesToPlot_family.LHS, rulesToPlot_family.RHS)]
rulesToPlot_family['sortedRow'] = rulesToPlot_family['sortedRow'].astype(str)
rulesToPlot_family.drop_duplicates(subset=['sortedRow'], inplace=True)

# Plot Graphically
rulesToPlot_family=rulesToPlot_family[:15]
fig = plt.figure(figsize=(17, 17)) 
G = nx.from_pandas_edgelist(rulesToPlot_family, 'LHS', 'RHS')
np.random.seed(1000)
nx.draw(G, with_labels=True, 
        node_size=30, 
        node_color="red", 
        pos=nx.spring_layout(G))
plt.axis('equal')
plt.show()

- **4.6) Modelling: By Family**
- 4.6.2) Key Takeaways<br>
<br>
- **Most frequent consequents:** Rice.
- All the dots that are closer to each of these "centers", are the most usual antecedents.
- The closer is the point, the higher is the confidence level.
- The two closest antecedents are: {Holds, Sizzling and Drinks} and {White Wine, Sizzling, Meat and Starters}.
- In the end, we decided do not take into account this results in our final recommendations, since we believe that products provide us a much more detailed explanation of costumers behaviors.

- **4.7) Menus Suggestions**
- 4.7.1) Menus for Dinne-Inns

- **Itemset:** "list of all the items in the antecedent and the consequent".
- **Support:** "the fraction of the total number of transactions in which the itemset occurs (%)". 
- **Antecedent Support:** represents the frequency of the antecedent itemset in the dataset (%).
- **Consequent Support:** represents the frequency of the consequent itemset in the dataset (%).
- **Confidence:** "the conditional probability of occurrence of consequent given the antecedent (%)".
- **Lift:** "the conditional probability of occurrence of consequent given the antecedent".

In [ ]:
# Measuring the quantity of products per set.

frequent_itemsets_dinne_in_no_water['products_per_set'] = frequent_itemsets_dinne_in_no_water['itemsets'].apply(lambda x: len(x))


# Checking the results for a 'products_per_set' = 2 and a Support >= 0.1.

# Reminding, a support repreents the percentage of transactions in the entire dataset that contain a particular itemset. 
# In this case, we aim to filter all the itemsets that represents at least 10% of the total transactions.

frequent_itemsets_dinne_in_no_water[(frequent_itemsets_dinne_in_no_water['products_per_set'] == 2) & 
                                    (frequent_itemsets_dinne_in_no_water['support'] >= 0.17)]

In [ ]:
# Itemsets with both high 'Confidence' and high 'Support'.
# This indicates that many customers purchase A and B together, 
# and customers who purchase A are highly likely to also purchase B.
# Together, a high support and high confidence for an association rule indicate that 
# the antecedent and consequent occur together frequently, 
# and that when the antecedent occurs, the consequent is highly likely to occur as well.

rules_support_dinne_in_no_water[(rules_support_dinne_in_no_water['confidence'] >= 0.60) &
                                   (rules_support_dinne_in_no_water['support'] >= 0.20)
                                  ]

In [ ]:
rules_support_dinne_in[(rules_support_dinne_in['confidence'] >= 0.90) &
                                   (rules_support_dinne_in['support'] >= 0.20)
                                  ]

In [ ]:
# Itemsets with both high 'Confidence' and high 'Lift'.

rules_confidence_dinne_in_no_water[(rules_confidence_dinne_in_no_water['lift'] >= 2.5) & 
                                   (rules_confidence_dinne_in_no_water['confidence'] >= 0.6)
                                  ]

In [ ]:
# Here we are assessing if there are products that customers tend not to buy together, 
# once they bought one of them.
# As we can see below, there are no substitute products in Dinne - Inn.

rules_lift_dinne_in_no_water[rules_lift_dinne_in_no_water.lift<=1]

**Relationships analysis | Suggestions for Dinne - Inns**
<br>
-  **Substitute Products:** there are no substitute products (since there is no negative lift), which means that we do not suggest any product deletion from the menu.  
-  **Creation of new products:** We suggest the creation of a new product of Noodles without meat. If there is a confidence of 100%, it means that when people order **"No Meat"**, they also order **"Noodles with meat"**, which makes sense. However, if we see the other way around, we check that 67% of the times, if the costumer order **"Noodles with meat"**, also order **"No meat"**, which is very relevant. Additionally, a lift of ~5.4, which means that the possibility of order **Noodles with meat** is 5.4 times higher than if they were independent.
-  **Creation of new menu 1:** **Spring roll** with **Egg fried rice** (support>0.2 and confidence>0.6) and with **Mineral water 1.5lt** (support>0.33 and confidence>0.94 with each of the previous refered products).
-  **Creation of new menu 2:** **Sweet sour chicken** with **Egg fried rice** (support>0.2 and confidence>0.6) and with **Mineral water 1.5lt** (support>0.28 and confidence>0.93 with each of the previous refered products).
-  **Increse Prices:** There is lift above 5 and a confidence above 0.6, on the items **Naan - Jira pulao** and **Chick tikka masala - Naan**. **Naan** is a starter, **jira pulao** is a side dish and **Chick tikka masala** a main indian dish. So, here there are two approaches we can follow: promote a discount on the the starter **Naan** and then, slightly increase the main and the side dish. Or, in case this leads to a decrease in the demand for main and side dish, we can also think about a menu 3.

<br>

**Additional Recommendations**
<br>
-  **Saturday Lunch Campaign:** We have noticed above that Sunday lunch, represents an outlier both in terms of orders volume but also in proportional to the dinner sales volume. However, we do no verify the same on Saturday. In this sense, we propose a possible campaign: if you order on Saturday Lunch before 4pm, we offer you a discount on next dinner visit.
- **Best Holidays Promotion:** The three most profitable holidays are separated by 13 days: 24.12.2018 (Christmas Eve), 31.12.2018 (New Year's Eve) and 06.01.2018 (Epiphany). We suggest the adoption of agressive marketing strategies for those specific days: first, the business hours can be slightly extended since we know that there must be high demand; second, we can create special products (for examples, a discount on the most expensive starters and desserts, since the in a special occasition, the costumers may be willing to spend more); and third, think about some partnership nearby the restaurant: festivals, cinema and massage center, for example.
- **Worst Holidays Promotion:** For the worst holidays, we suggest a campaign to attract costumers. For example: if you come on one of this holidays, you receive a 50% discount on the next visit in May or June. The holidays are the following: 1.05.2018 (Labour Day/May Day) and (28.05.2018 (Orthodox Pentecost Monday)).

- **4.7) Menus Suggestions**
- 4.7.2) Menus for Deliveries

- **Itemset:** "list of all the items in the antecedent and the consequent".
- **Support:** "the fraction of the total number of transactions in which the itemset occurs (%)". 
- **Antecedent Support:** represents the frequency of the antecedent itemset in the dataset (%).
- **Consequent Support:** represents the frequency of the consequent itemset in the dataset (%).
- **Confidence:** "the conditional probability of occurrence of consequent given the antecedent (%)".
- **Lift:** "the conditional probability of occurrence of consequent given the antecedent".

In [ ]:
# Measuring the quantity of products per set.

frequent_itemsets_delivery['products_per_set'] = frequent_itemsets_delivery['itemsets'].apply(lambda x: len(x))


# Checking the results for a 'products_per_set' = 2 and a Support >= 0.1.
# Reminding, a support repreents the percentage of transactions in the entire dataset that contain a particular itemset. 
# In this case, we aim to filter all the itemsets that represents at least 10% of the total transactions.

frequent_itemsets_delivery[(frequent_itemsets_delivery['products_per_set'] == 2) & 
                                    (frequent_itemsets_delivery['support'] >= 0.1)
                                   ]

rules_support_delivery[(rules_support_delivery['support'] >= 0.2) & 
                                   (rules_support_delivery['confidence'] >= 0.50)]

In [ ]:
# Itemsets with both high 'Confidence' and high 'Lift'.
rules_confidence_delivery[(rules_confidence_delivery['lift'] >= 1.5) & 
                                   (rules_confidence_delivery['confidence'] >= 0.5)]

In [ ]:
# Here we are assessing if there are products that customers tend not to buy together, 
# once they bought one of them.
# As we can see below, there are no substitute products in Deliveries

rules_lift_delivery[rules_lift_delivery.lift<=1]

**Relationships analysis | Suggestions for Deliveries**
<br>
-  **Substitute Products:** there are no substitute products (since there is no negative lift), which means that we do not suggest any product deletion from the menu.  
-  **Creation of new products:** We suggest the creation of a new product of Noodles without meat. If there is a confidence of 100%, it means that when people order **"No Meat"**, they also order **"Noodles with meat"**, which makes sense. Additionally, a lift of ~8, which means that the possibility of order **Noodles with meat** is ~8 times higher than if they were independent.
-  **Creation of new menu 1:** **Sweet sour chicken** with **Egg fried rice** (support>0.2 and confidence>0.6).
-  **Increse Prices 1:** There is lift above 4.5 and a confidence above 0.54 on the items **Jira pulao - Naan**. **Naan** is an indian starter, **jira pulao** is an indian side dish. So, here, in opposition to what we suggested in dinne-in, there is no main dish so that we could link the products. Anyway, promote a discount on the the starter **Naan** and increase the price of the side dish could be a possibility. 
-  **Increse Prices 2:** There is lift above 8.2 and a confidence above 0.70 on the items **extra pancakes - extra sauce**, naturally. Here, we could explore a promotion on the price of additional **pancakes**, along with an increase in the **extra source** price, for example. 


**Additional Recommendations**
<br>
-  **Saturday Lunch Campaign:** We have noticed above that Sunday lunch, represents an outlier both in terms of orders volume but also in proportional to the dinner sales volume. However, we do no verify the same on Saturday. In this sense, we propose a possible campaign: if you order on Saturday Lunch before 4pm, we offer you a discount on next dinner visit.
- **Best Holidays Promotion:** The three most profitable holidays are separated by 13 days: 24.12 (Christmas Eve), 31.12 (New Year's Eve) and 06.01 (Epiphany). We suggest the adoption of agressive marketing strategies for those specific days: first, the business hours can be slightly extended since we know that there must be high demand; second, we can create special products (for examples, a discount on the most expensive starters and desserts, since the in a special occasition, the costumers may be willing to spend more); and third, think about some partnership nearby the restaurant: festivals, cinema and massage center, for example.
- **Worst Holidays Promotion:** For the worst holidays, we suggest a campaign to attract costumers. For example: if you come on one of this holidays, you receive a 50% discount on the next visit in May or June. The holidays are the following: 1 of May (Labour Day/May Day) and 28 of May (Orthodox Pentecost Monday).

# **5. Deployment**


- **5.1) Final Recommendation System for Dinne-Inns**
- **5.2) Final Recommendation System for Deliveries**

- **5.1) Final Recommendation System for Dinne-Inns**

In [ ]:
# Define the order as a dinner in case the invoice was issued after 6pm.
df["Dinner_meal"] = [0 if x.hour < 18 else 1 for x in df.InvoiceDateHour]

# Creating the bins for the number of people per order.
df["People_agg"] = ["1" if i == 1 else 
                    "2" if i == 2 else 
                    "3" if i == 3 else 
                    "4-5" if i <= 5 else 
                    "5-10" if i <= 10 else 
                    "10+" for i in df.Pax ]

# Check sizes of customer segments
df_dinne_inns = df[df.IsDelivery == 0]
ct = pd.crosstab(index=[df_dinne_inns.People_agg],
                 columns=[df_dinne_inns.Dinner_meal, df_dinne_inns.Holiday])

# Sort by People_agg
ct_sorted = ct.sort_index(level=0)
ct_sorted

In [ ]:
# Auxiliar functions:
    # 1. df_people
    # 2. df_products_alphabetically
    # 3. df_rules

    
# 1.This function returns the transactions dataset filtered by the following variables: 
    # 'Deliveries', 
    # 'People_agg'
    # 'Dinner_meal'.
    # 'Holidays'
def df_people(People_agg):
    
    # 1.1 Obtain the bins for people category.
    if People_agg <= 1: 
        People = "1"
    elif People_agg <=2:
        People = "2"
    elif People_agg <= 5:
        People = "3-5"
    elif People_agg <= 10:
        People = "5-10"
    else:
        People = "10+"
        
    # 1.2 Get current time and check if dinner.
    dinner = 1                      # By default, we are assuming that the meal is a dinner, since the majority of the meals are dinner.
    if dt.datetime.now().hour < 18: # By previously looking at the plots, we have defined this as our threshold.
        dinner = 0 
        
    # 1.3 Get current date and check if holiday
    holiday_day = 0
    if len(df[(df.Holiday == 1) & (df.month == dt.datetime.now().month) & (df.day == dt.datetime.now().day)]) != 0:
        holiday_day = 1
    
    # 1.4 Making our final filter, storing in a new variable, called df_finne_inns. 
    df_dinne_inns = df[(df.IsDelivery == 0) & 
                       (df.Dinner_meal == dinner) & 
                       (df.Holiday == holiday_day) &
                       (df.People_agg == People)].copy(deep=True)
    return df_dinne_inns


# 2. This function returns a list of alphabetically sorted strings with all possible itemset combinations
def df_productset_alphabetically(products):
    
     # Get all possible combinations as a list of tuples.
    product_sets = []
    for i in range(1,len(products) + 1):
        for i in itertools.combinations(products, i):
            product_sets.append(i)

    # Transform tuples into list of strings, sort the list alphabetically and create string out of sorted list .
    productslists = [list(x) for x in product_sets]
    for i in productslists:
        i.sort()
    productsstrings = [" ".join(x) for x in productslists]
    return productsstrings


# 3. This functions returns a dataframe with rules, based on the df returned by filter_df(pax)
def df_rules(People_agg, support_min, confidence_min):
    
    # Create frequent itemset of filtered df
    df_dinne_inn = pd.pivot_table(df_people(People_agg)[["DocNumber","ProductDesignation"]], 
                            index='DocNumber', 
                            columns='ProductDesignation', 
                            aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0) 
    frequent_itemsets_df_dinne_inn = apriori(df_dinne_inn, min_support=support_min, use_colnames=True) 
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence_min)
    
    # Remove duplicate if reversed rules
    rules['LHS'] = [','.join(list(x)) for x in rules['antecedents']]
    rules['RHS'] = [','.join(list(x)) for x in rules['consequents']]
    rules['sortedRow'] = [sorted([a,b]) for a,b in zip(rules.LHS, rules.RHS)]
    rules['sortedRow'] = rules['sortedRow'].astype(str)
    rules.drop_duplicates(subset=['sortedRow'], inplace=True)

    # Transform antecedents and consequents into list of strings, sort the list alphabetically and create string out of sorted list 
    rules["antecedents"] = rules["antecedents"].apply(lambda x: list(x))
    rules["consequents"] = rules["consequents"].apply(lambda x: list(x))

    for i in rules.index.values:
        rules.loc[i,"antecedents"].sort()
        rules.loc[i,"consequents"].sort()

    rules["antecedents"] = rules["antecedents"].apply(lambda x: " ".join(x))
    rules["consequents"] = rules["consequents"].apply(lambda x: " ".join(x))

    # Remove space in the beginning of antecedents and consequents
    rules["antecedents"] = rules["antecedents"].apply(lambda x: x[1:] if x[0] == " " else x)
    rules["consequents"] = rules["consequents"].apply(lambda x: x[1:] if x[0] == " " else x)
    
    return rules

In [ ]:
# Get recommended products 
# Based on number of persons and ordered items, return maximum 7 itemsets with the highest lift

def final_recommendation_dinne_inn(People_agg, products, support_min=0.05, confidence_min=0.15, lift_min=1.1):
    productsstrings = df_productset_alphabetically(products)
    rules = df_rules(People_agg, support_min, confidence_min)
    
    # Filter possible recommendations
    suggestions = rules[rules.antecedents.isin(productsstrings)]
    
    # Sort by lift and select relevant information of top ten
    best_seven = suggestions.sort_values("lift", ascending=False)[:7][["antecedents", "consequents", 
                                                                   "support", "confidence", "lift"]]
    
    # Only consider rules with certain lift 
    best_seven = best_seven[best_seven.lift >= lift_min]
    
    return best_seven

In [ ]:
# Example: 
    # Two costumers that orders "Mineral water 1.5lt": 
        # "What are the main suggestions under our model?"

final_recommendation_dinne_inn(2, ["Mineral water 1.5lt"])

- **5.2) Final Recommendation System for Deliveries**

In [ ]:
# Define the order as a dinner in case the invoice was issued after 6pm.
df["Dinner_meal"] = [0 if x.hour < 18 else 1 for x in df.InvoiceDateHour]

# Check sizes of customer segments
df_deliveries = df[df.IsDelivery == 1]
pd.crosstab(df_deliveries.CustomerCity,[df_deliveries.Dinner_meal, df_deliveries.Holiday])
# Sort by People_agg
ct_sorted_deliveries = ct.sort_index(level=0)
ct_sorted_deliveries

In [ ]:
# Auxiliar functions:
    # 1. df_city_delivery
    # 2. df_products_alphabetically_delivery
    # 3. df_rules_delivery

# 1.This function returns the transactions dataseet filtered by:
    # Only Delivery customers
    # Corresponding city
    # Whether is Holiday or not
    # Whether is Dinner or not
def df_city_delivery(city):
    # Get current time and check if dinner
    dinner_meal = 1
    if dt.datetime.now().hour < 18:
        dinner_meal = 0
        
    # Get current date and check if holiday
    holiday_day = 0
    if len(df[(df.Holiday == 1) & (df.month == dt.datetime.now().month) & (df.day == dt.datetime.now().day)]) != 0:
        holiday_day = 1
    
    # Filter dataset 
    df_deliveries_1 = df[(df.IsDelivery == 1) & 
                         (df.Dinner_meal == dinner_meal) & 
                         (df.Holiday == holiday_day) & 
                         (df.CustomerCity == city)].copy(deep=True)
    if len(df_deliveries_1.index.values) > 75:
        return df_deliveries_1
    else: 
        return df[(df.IsDelivery == 1) & 
                  (df.Dinner_meal == dinner_meal) & 
                  (df.Holiday == holiday_day)].copy(deep=True)

# 2. This function returns a list of alphabetically sorted strings with all possible itemset combinations
def df_products_alphabetically_delivery(products):
    
     # Get all possible combinations as a list of tuples
    productsets = []
    for i in range(1,len(products) + 1):
        for i in itertools.combinations(products, i):
            productsets.append(i)

    # Transform tuples into list of strings, sort the list alphabetically and create string out of sorted list 
    productslists = [list(x) for x in productsets]
    for i in productslists:
        i.sort()
    productsstrings = [" ".join(x) for x in productslists]
    return productsstrings

# 3. This functions returns a dataframe with rules, based on the df returned by filter_df(pax)
def df_rules_delivery(city, support_min, confidence_min):
    
    # Create frequent itemset of filtered df
    df_deliveries = pd.pivot_table(df_city_delivery(city)[["DocNumber","ProductDesignation"]], 
                            index='DocNumber', 
                            columns='ProductDesignation', 
                            aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0) 
    frequent_itemsets = apriori(df_deliveries, min_support=support_min, use_colnames=True) 
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence_min)
    
    # Remove duplicate if reversed rules
    rules['LHS'] = [','.join(list(x)) for x in rules['antecedents']]
    rules['RHS'] = [','.join(list(x)) for x in rules['consequents']]
    rules['sortedRow'] = [sorted([a,b]) for a,b in zip(rules.LHS, rules.RHS)]
    rules['sortedRow'] = rules['sortedRow'].astype(str)
    rules.drop_duplicates(subset=['sortedRow'], inplace=True)

    # Transform antecedents and consequents into list of strings, sort the list alphabetically and create string out of sorted list 
    rules["antecedents"] = rules["antecedents"].apply(lambda x: list(x))
    rules["consequents"] = rules["consequents"].apply(lambda x: list(x))

    for i in rules.index.values:
        rules.loc[i,"antecedents"].sort()
        rules.loc[i,"consequents"].sort()

    rules["antecedents"] = rules["antecedents"].apply(lambda x: " ".join(x))
    rules["consequents"] = rules["consequents"].apply(lambda x: " ".join(x))

    # Remove space in the beginning of antecedents and consequents
    rules["antecedents"] = rules["antecedents"].apply(lambda x: x[1:] if x[0] == " " else x)
    rules["consequents"] = rules["consequents"].apply(lambda x: x[1:] if x[0] == " " else x)
    
    return rules

In [ ]:
# Get recommended products 
# Based on number of city and ordered products, return maximum 7 itemsets with the highest lift.
def final_recommendation_delivery(city, products, support_min=0.03, confidence_min=0.1, lift_min=1.1):
    productsstrings_delivery = df_products_alphabetically_delivery(products)
    rules_delivery = df_rules_delivery(city, support_min, confidence_min)
    
    # Filter possible recommendations
    suggestions = rules_delivery[rules_delivery.antecedents.isin(productsstrings_delivery)]
    
    # Sort by lift and select relevant information of top ten
    best_seven = suggestions.sort_values("lift", ascending=False)[:7][["antecedents", "consequents", 
                                                                        "support", "confidence", "lift"]]
    
    # Only consider rules with certain lift 
    best_seven = best_seven[best_seven.lift >= lift_min]
    
    return best_seven

In [ ]:
# Example: 
    # A costumer that orders "Spring roll" in "Egkomi":
        # "What are the main suggestions under our model?"

final_recommendation_delivery("Egkomi", ["Spring roll"])